<a href="https://colab.research.google.com/github/patrickvonplaten/notebooks/blob/master/Getting_the_most_out_of_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting the most out of LLMs

Large Language Models (LLMs) like ChatGPT, Falcon, and LLama are rapidly advancing in their ability to tackle human-centric tasks, establishing themselves as essential tools in modern knowledge-based industries.
Deploying these models in real-world tasks remains challenging however:

- In order to exhibit near-human text understanding and generation capabilities, LLMs currently require to be composed of billions of parameters (see [Kaplan et. al](https://arxiv.org/abs/2001.08361), [Wei et. al](https://arxiv.org/abs/2206.07682)). This consequently amplifies the memory demands for inference.
- In many real-world tasks, LLMs need to be given a extensive contextual information. This necessitates the model's capability to manage very long input sequences during inference.

The crux of these challenges lies in augmenting the computational and memory capabilities of LLMs, especially when handling expansive input sequences.

In this blog post, we will go over the most effective techniques to tackle these challenges. for efficient LLM deployment:

1. **Lower Precision**: Research has shown that operating at reduced numerical precision, namely 8bit and 4bit, can achieve computational advantages without a considerable decline in model performance.

2. **Flash Attention:** Flash Attention is a variation of the attention algorithm that not only provides a more memory-efficient approach but also realizes increased efficiency due to optimized GPU memory utilization.

3. **Architectural Innovations:** Considering that LLMs are always depoyed in the same way during inference, namely autoregressive text generation with a long input context, specialized model architectures have been proposed that allow for more efficient inference. The most important advancement in model architectures hereby are: [Alibi](https://arxiv.org/abs/2108.12409), [Rotary  embeddings](https://arxiv.org/abs/2104.09864) and [Multi-Query Attention (MQA)](https://arxiv.org/abs/1911.02150).

Throughout this piece, we will offer an analysis of auto-regressive generation from a tensor's perspective, delve into the pros and cons of adopting lower precision, and provide a comprehensive exploration of the latest attention mechanisms and architectural developments. While doing so, we run practical examples showcasing each of the feature improvements explained above.

## 1. Harnessing the Power of Lower Precision

Memory requirements of LLMs can be best understood by seeing the LLM as a set of weight matrices and vectors and the text inputs as a sequence of vectors. In the following, the definition *weights* will be used to signify all model weight matrices and vectors.

At the time of writing this paper, LLMs consists of at least a couple billion parameters mearning that the sum of all entries in all weights is larger than 1,000,000,000. Each entry thereby consists of a decimal number, e.g. `4.5689` which is usually stored in either [float32](https://en.wikipedia.org/wiki/Single-precision_floating-point_format), [bfloat16](https://en.wikipedia.org/wiki/Bfloat16_floating-point_format), or [float16](https://en.wikipedia.org/wiki/Half-precision_floating-point_format) format. This allows us to easily compute the memory requirement to load the LLM into memory:

> *Loading the weights of a model having X billion parameters requires roughly 4 * X GB of VRAM in float32 precision*

Nowadays, models are however rarely trained in full float32 precision, but usually in bfloat16 precision or less frequently in float16 precision. Therefore the role of thumb becomes:

> *Loading the weights of a model having X billion parameters requires roughly 2 * X GB of VRAM in bfloat16/float16 precision*

For shorter text inputs (less than 1024 tokens), the memory requirement for inference is very much dominated by the memory requirement to load the weights. Therefore, for now let's assume that the memory requirement for inference is simply the memory requirement to load the model into the GPU VRAM.

To give some examples of how much VRAM it roughly takes to load a model in bfloat16:
- **GPT3** requires 2 * 175 GB = **350 GB** VRAM
- [**Bloom**]() requires 2 * 176 GB = **352 GB** VRAM
- [**Llama-2-70b**]() requires 2 * 70 GB = **140 GB** VRAM
- [**Falcon-40b**]() requires 2 * 40 GB = **80 GB** VRAM
- [**MPT-30b**]() requires 2 * 30 GB = **60 GB** VRAM
- [**bigcode/starcoder**]() requires 2 * 15.5 = **31 GB** VRAM

As of writing this document, the largest GPU chip on the market is the A100 offering 80GB of VRAM. Most of the models listed before require more than 80GB just to be loaded and therefore necessarily require [tensor parallelism](https://huggingface.co/docs/transformers/v4.15.0/parallelism#tensor-parallelism) and/or [pipeline parallelism](https://huggingface.co/docs/transformers/v4.15.0/parallelism#naive-model-parallel-vertical-and-pipeline-parallel)

🤗 Transformers does not support tensor parallelism out of the box as it requires the model architecture to be written in a specific way. If you're interested in writing models in a tensor-parallelism friendly way, feel free to have a look at [the text-generation-inference library](https://github.com/huggingface/text-generation-inference/tree/main/server/text_generation_server/models/custom_modeling).

Pipeline parallelism is supported out-of-the-box. For this, simply load the model with `device="auto"` which will automatically place the different layers on the available GPUs.

If you are working with a 8 x 80GB A100 node, you could load BLOOM as follows

In [ ]:
!pip install transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 18.2 MB/s eta 0:00:00


In [ ]:
# from transformers import AutoModelForCausalLM

# model = AutoModelForCausalLM.from_pretrained("bigscience/bloom", device="auto")

which would equally distribute the attention layers automatically over all available GPUs.

Throughout this notebook, we will use [bigcode/octocoder](https://huggingface.co/bigcode/octocoder) as it can be run on a single 40 GB A100 GPU device chip. Note that all memory and speed optimizations that we will apply going forward, are equally applicable to models that require model or tensor parallelism.

Great, remembering our rule of thumb above we would expect the memory requirement to run inference with `bigcode/octocoder` to be around 31 GB VRAM. Let's give it a try.

We first load the model and tokenizer and then pass both to Transformers' [pipeline](https://huggingface.co/docs/transformers/main_classes/pipelines).

The model is loaded in *bloat16* precision.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

model = AutoModelForCausalLM.from_pretrained("bigcode/octocoder", torch_dtype=torch.bfloat16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("bigcode/octocoder")

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
prompt = "Question: Please write a function in Python that transforms bytes to Giga bytes.\n\nAnswer:"

result = pipe(prompt, max_new_tokens=60)[0]["generated_text"][len(prompt):]
result

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


' Here is a Python function that transforms bytes to Giga bytes:\n\n```python\ndef bytes_to_giga_bytes(bytes):\n    return bytes / 1024 / 1024 / 1024\n```\n\nThis function takes a single'

Nice, we can now directly use the result to compute how much Giga Bytes were needed.

In [ ]:
def bytes_to_giga_bytes(bytes):
  return bytes / 1024 / 1024 / 1024

In [ ]:
bytes_to_giga_bytes(torch.cuda.max_memory_allocated())

29.0260648727417

Close enough to our back-of-the-envelop computation! We can see the number is not exactly correct as going from bytes to kilobytes etc... requires multiplication of 1024 instead of 1000. Therefore the back-of-the-envelope formula can also be understood as a "at most X GB" computation.

Let's free some memory for the next experiments.

In [ ]:
del pipe
del model

import gc
import torch

def flush():
  gc.collect()
  torch.cuda.empty_cache()
  torch.cuda.reset_peak_memory_stats()

In [ ]:
flush()

As we can see the memory requirement is roughly what we expected. Note that if we would have tried to run the model in full float32 precision, a whooping 64 GB of VRAM would have been required.

> Almost all models are trained in bfloat16 nowadays, there is no reason to run the model in full float32 precision if [your GPU supports bfloat16](https://discuss.pytorch.org/t/bfloat16-native-support/117155/5). Float32 won't give better inference results than the precision that was used to train the model.

What if your GPU does not have 32 GB of VRAM? It has been found that model weights can be quantized to 8bit or 4bits without a significant loss in performance (see [Dettmers et al.](https://arxiv.org/abs/2208.07339)).

It is possible to quantize the models even further - to 3 or even 2 bits with an acceptable loss in performance as shown in the recent [GPTQ paper](https://huggingface.co/docs/transformers/main_classes/quantization#general-usage) 🤯.

Without going into too many details, quantization schemes aim at reducing the precision of weights while trying to keep the model's inference results as accurate as possible (*a.k.a* as close as possible to bfloat16).
Note that quantization works especially well for text generation since all we care about is choosing the *most likely next token* and don't really care about the exact values of the next token logit distribution. All that matters is that the the next token logit distribution stays roughly the same so that an `argmax` or `topk` operation gives the same results.

There are various quantization techniques, which we won't discuss in detail here, but in general all quantization techniques work as follows:
- 1. Quantize all weights to the target precision
- 2. Load the quantized weights, and pass the input sequence of vectors in bfloat16 precision
- 3. Dynamically dequantize weights to bfloat16 to perform the computation with their input vectors in bfloat16 precision
- 4. Quantize the weights again to the target precision after computation with their inputs.

In a nutshell, this means that that input-matrix multiplications, with $X$ being the *inputs*, $W$ being a single weight matrix and $Y$ being the output:

$Y = X * W$

are changed to

$Y = X * \text{dequantize}(W); \text{quantize}(W)$

for every matrix multiplication.

Therefore, inference time is often **not** reduced when using quantized weights, but rather increases.

Enough theory, let's give it a try! In Transformers, you can very easily load weights as follows.

Make sure that the [`bitsandbytes`](https://github.com/TimDettmers/bitsandbytes) library needs to be installed.

In [ ]:
# !pip install bitsandbytes

and then we can load models in 8bit quantization by simply adding the `load_in_8bit` flag:

In [ ]:
model = AutoModelForCausalLM.from_pretrained("bigcode/octocoder", load_in_8bit=True, low_cpu_mem_usage=True)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Now, let's run our example again and measure the memory usage.

In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

result = pipe(prompt, max_new_tokens=60)[0]["generated_text"][len(prompt):]
result

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': 'Question: Please write a function in Python that transforms bytes to Giga bytes.\n\nAnswer: Here is a Python function that transforms bytes to Giga bytes:\n\n```python\ndef bytes_to_giga_bytes(bytes):\n    return bytes / 1024 / 1024 / 1024\n```\n\nThis function takes a single'}]

Nice, we're getting the same result as before, so no loss in accuracy! Let's look at how much memory was used this time.

In [ ]:
bytes_to_giga_bytes(torch.cuda.max_memory_allocated())

17.503968238830566

Significantly less, we're down to just 17.5 GBs and could therefore run this model on consumer GPUs like the 4090. Let's see what 4bit quantization produces.

In [ ]:
del model
del pipe

In [ ]:
flush()

Nice, we're seeing a very nice gain in memory efficiency and more or less no degradation to the model's output. However we can also notice a slight slow-down during inference.

4bit quantization can be loaded in the same way, by just passing `load_in_4bit=True` instead of `load_in_8bit=True`.



In [ ]:
model = AutoModelForCausalLM.from_pretrained("bigcode/octocoder", load_in_4bit=True, low_cpu_mem_usage=True)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

result = pipe(prompt, max_new_tokens=60)[0]["generated_text"][len(prompt):]
result

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': 'Question: Please write a function in Python that transforms bytes to Giga bytes.\n\nAnswer: Here                                                                               '}]

We see that this time the generate function prematurely stopped, showing a loss in accuracy! Let's see how much memory was required.

In [ ]:
bytes_to_giga_bytes(torch.cuda.max_memory_allocated())

9.543574333190918

In [ ]:
del model
del pipe

In [ ]:
flush()

Just 9.5GB! That's really not much for a ~16 billion parameter model, but we see that 4bit quantization often comes at the cost of a loss in accuracy.

Running OctoCoder in 4bit reduces the required GPU VRAM from 32GB to just a bit over 9GB which is very low for a 15.5 billion parameter model. 4bit quantization allows the model to be run on GPUs such as RTX3090, V100 and T4 which are quite accesible for most people.

To quantize the model even further, we recommend looking into the [`AutoGPTQ`](https://huggingface.co/docs/transformers/main/en/main_classes/quantization#autogptq-integration`) implementation.

> Overall, it is important to remember that model quatization trades improved memory efficiency against accuracy and in most cases inference time.

If GPU memory is not a constraint for your use case, there is often no need to look into quantization. However many GPUs simply can't run LLMs without quantization methods and in this case 4bit and 8bit quantization schemes are extremely useful tools.

For more in-detail usage information, we strongly recommend to take a look at the [Transformers Quantization Docs](https://huggingface.co/docs/transformers/main_classes/quantization#general-usage) .

Next, let's look into how we can improve computational and memory efficiency by using better algorithms and an improved model architecture.

# 2. Flash Attention: A Leap Forward

Today's top-performing LLMs share more or less the same fundamental architecture that consists of feed-forward layers, activation layers, layer normalization layers, and most crucially, self-attention layers.

Self-attention layers are central to Large Language Models (LLMs) in that they are enabling the model to understand the contextual relationships between input words (or more specifically tokens).

> The catch however is that of all the layers self-attention layers grow *quadratically( both in time and memory complexity with the sequence length $N$.

While this is not really noticable for shorter input sequences up to 1000 input tokens, it becomes a serious problem for longer input sequences of around 16000 input tokens.

Let's take a closer look. The formula to compute the output $\mathbf{O}$ of a self-attention layer for an input $\mathbf{X}$ of length $N$ is:

$$ \textbf{O} = \text{Attn}(\mathbf{X}) = \mathbf{V} \times \text{Softmax}(\mathbf{QK}^T) \text{ with } \mathbf{Q} = \mathbf{W}_q \mathbf{X}, \mathbf{V} = \mathbf{W}_v \mathbf{X}, \mathbf{K} = \mathbf{W}_k \mathbf{X} $$

$\mathbf{X} = (\mathbf{x}_1, ... \mathbf{x}_{N})$ is thereby the input sequence to the attention layer. The projections $\mathbf{Q}$ and $\mathbf{K}$ will therefore also each consist of $N$ vectors resulting in the $\mathbf{QK}^T$ being of size $N^2$.

LLMs usually have multiple attention heads, thus doing multiple such computations in parallel.
Assuming, the LLM has 80 attention heads and runs in bfloat16 precision, we can calculate the memory requirement to store the $\mathbf{QK^T}$ matrices to be $80 * 2 * N^2$ bytes. For $N=1000$ only around 0.1 GB of VRAM are needed, however for $N=16000$ we would need 38 GB of VRAM and for $N=100,000$ we would need >1TB just to store the $\mathbf{QK}^T$ matrices.

Long story short, the default self-attention algorithm quickly becomes prohibitely memory expensive for large input contexts.

As LLMs improve in text comprehension and generation, they are applied to increasingly complex tasks. While models once handled the translation or summarization of a few sentences, they now manage entire pages, demanding the capability to process extensive input lengths.

How can we get rid of the exorbitant memory requirements for large input lengths? We need a new way to compute the self-attention mechanism that gets rid of the $QK^T$ matrix. [Tri Dao et al.](FlashAttention: Fast and Memory-Efficient Exact Attention with IO-Awareness) developed excatly such a new algorithm and called it *Flash Attention**.

In a nutshell, Flash Attention breaks the $ \mathbf{V} \times \text{Softmax}(\mathbf{QK}^T) $ computation apart and instead computes smaller smaller chunks of the output by iterating oven multiple softmax computation steps:

$$ \textbf{O}_i \leftarrow s^a_{ij} * \textbf{O}_i + s^b_{ij} * \mathbf{V}_{j} \times \text{Softmax}(\mathbf{QK}^T_{i,j}) \text{ for multiple } i, j \text{ iterations} $$

with $s^a_{ij}$ and $s^b_{ij}$ being some softmax normalization statistics that need to be recomputed for every $i$ and $j$.

Please note that the whole Flash Attention is a bit more complex and is greatly simplified here as going in too much depth is out of scope for the document. The reader is invited to take a look at the well-written [Flash Attention paper](https://arxiv.org/pdf/2205.14135.pdf) for more details.

The main take-away here is that:
> By keeping track of softmax normalization statistics and by using some smart mathematics, Flash Attention gives **numerical identical** outputs compared to the default self-attention layer at a memory cost that only increases linearly with $N$.

From looking at the formula, one would intuitively say that Flash Attention must be much slower compared to the default self-attention formula as more computation needs to be done. Indeed Flash Attention requires more FLOPs compared to normal attenion as the softmax normalization statistics have to constantely be recomputed.

> However Flash Attenion is much faster in inference compared to default attention which comes from its ability to significantly reduce the demands on the slower, high-bandwidth memory of the GPU (VRAM), focusing instead on the faster on-chip memory (SRAM).

Essentially, Flash Attention makes sure that all intermediate write and read operations can be done using the fast *on-chip* SRAM memory instead of having to access the VRAM memory.

Therefore there is absolutely no reason to **not** use Flash Attention if available.

Let's look at a practical example.

Our OctoCoder model now gets a significantly longer input prompt which includes a so-called system prompt*. System prompts are used to steer the LLM into a better assistant tailored to the users' task. In the following, we use a system prompt that will make OctoCoder a better coding assistant.

In [ ]:
system_prompt = """Below are a series of dialogues between various people and an AI technical assistant.
The assistant tries to be helpful, polite, honest, sophisticated, emotionally aware, and humble-but-knowledgeable.
The assistant is happy to help with code questions, and will do its best to understand exactly what is needed.
It also tries to avoid giving false or misleading information, and it caveats when it isn't entirely sure about the right answer.
That said, the assistant is practical and really does its best, and doesn't let caution get too much in the way of being useful.

The Starcoder models are a series of 15.5B parameter models trained on 80+ programming languages from The Stack (v1.2) (excluding opt-out requests).
The model uses Multi Query Attention, was trained using the Fill-in-the-Middle objective and with 8,192 tokens context window for a trillion tokens of heavily deduplicated data.

-----

Question: Write a function that takes two lists and returns a list that has alternating elements from each input list.

Answer: Sure. Here is a function that does that.

def alternating(list1, list2):
   results = []
   for i in range(len(list1)):
       results.append(list1[i])
       results.append(list2[i])
   return results

Question: Can you write some test cases for this function?

Answer: Sure, here are some tests.

assert alternating([10, 20, 30], [1, 2, 3]) == [10, 1, 20, 2, 30, 3]
assert alternating([True, False], [4, 5]) == [True, 4, False, 5]
assert alternating([], []) == []

Question: Modify the function so that it returns all input elements when the lists have uneven length. The elements from the longer list should be at the end.

Answer: Here is the modified function.

def alternating(list1, list2):
   results = []
   for i in range(min(len(list1), len(list2))):
       results.append(list1[i])
       results.append(list2[i])
   if len(list1) > len(list2):
       results.extend(list1[i+1:])
   else:
       results.extend(list2[i+1:])
   return results

-----
""""

 To achieve a critrical long enough input sequence to showcase the memory gains of Flash Attention, we will simply duplicate the system prompt by 5 and add our coding question from above.

In [ ]:
long_prompt = 5 * system_prompt + prompt

Let's instantiate our model again in bfloat16 precision

In [ ]:
model = AutoModelForCausalLM.from_pretrained("bigcode/octocoder", torch_dtype=torch.bfloat16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("bigcode/octocoder")

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

and run it to gauge the memory requirement when using vanilla attention.

In [ ]:
import time

start_time = time.time()
result = pipe(long_prompt, max_new_tokens=60)[0]["generated_text"][len(long_prompt):]

print(f"Generated in {time.time() - start_time} seconds.")
result

For comparison, let's run the same function, but enabling Flash Attention instead.

In [ ]:
# TODO: Wait for Flash Attn 2?

## 3. The Science Behind LLM Architectures: Strategic Selection for Long Text Inputs and Chat

So far we have looked into improving computational and memory efficiency by:
- Casting the weights to a lower precision format
- Improving the self-attention algorithm with a more memory- and compute efficient algorithm

Let's now look into how we can change the architecture of an LLM so that it is most effective and effecient for:
- Tasks requiring the LLM to handle long text inputs, such as retrieval augmented Questions Answering, Summarization, ...
- Chat

Note that *Chat* also requires the model to handle long text iputs, but in addition it necessitates that the model is able to effeciently handle the back-and-forth dialogue between user and assistant (such as ChatGPT).

Once trained, the fundamental LLMs architecture is difficult to change, so it is important to make considerations about the LLMs tasks beforehand and accordingly optimize the models architecture.

There are two important components of the model architecture that quickly become memory and/or performance bottlenecks for large input sequences and chat.

- The positional embeddings
- The k/v cache

Let's go over each component in more-detail

### 3.1 Improving positional embeddings of LLMs

Self-attention puts each token into relation with each other token.
As an example, the $\text{Softmax}(\mathbf{QK}^T)$ matrix of the text input sequence _"Hello", "I", "love", "you"_ could look as follows:

![](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/self_attn_tokens.png)

Each word token is given a probalitiy mass at which it attends all other word tokens, therefore is put into relation with all other word tokens. E.g. the word _"love"_ attends to the word _"Hello"_ with 0.05%, to _"I"_ with 0.3%, and to itself with 0.65%.

A LLM based on self-attention, but without position embeddings would have great difficulties to understand the positions of the text inputs to each other. Due to self-attention, each word token appears to have the same distance from all others. This perception is based on the probability score computed by $\mathbf{QK}^T$ which relates each word token to each other word token in $O(1)$ computations regardless of their relative positional distance to each other. As a result, A LLM without positional embeddings would have a hard time understanding sentence order, *e.g.* differentiating between _"Hello I love you"_ and _"You love I hello_.

To remedy this problem, the authors of [*Attention Is All You Need*](https://arxiv.org/abs/1706.03762) paper introduced sinusoidal positional embeddings $\mathbf{P} = \mathbf{p}_1, \ldots, \mathbf{p}_N$ that were simply added to the input sequence $\mathbf{\hat{X}} = \mathbf{\hat{x}}_1, \ldots, \mathbf{\hat{x}}_N$ = $\mathbf{x}_1 + \mathbf{p}_1, \ldots, \mathbf{x}_N + \mathbf{x}_N $. Each $\mathbf{p}_i$ therefore has a unique signature that purely depends on its position $i$ and therefore can cue the model to better learn sentence order.

Instead of using fixed sinuoidal position embeddings, other work such as [Devlin et al.](https://arxiv.org/abs/1810.04805) used learned embeddings to let the model learn the positional embeddings $P$.

Sinoidal and learned position embeddings were the predominant methods to encode sentence order into LLMs, but problems have been found:

- 1.) Sinoidal and learned position embeddings are both absolute positional embeddings, *i.e.* encoding a unique embedding for each position id: $0, \ldots, N$. As has been found by [Huang et al.](https://arxiv.org/abs/2009.13658) and [Su et al.](https://arxiv.org/abs/2104.09864)], absolute positional embeddings lead to poor LLM performance for long text inputs. For long text inputs, it is advantageous if the model learns the relative positional distance input text tokens have to each other instead of their absolute position.

- 2.) Learned position embeddings have to be trained on a fixed input length $N$, making it difficult to extrapolate to input length longer than what it was trained on.

Recently, relative positonal embeddings that can tackle the problems mentioned before have become more popular, most notably:
- [Rotary Position Embedding (RoPE)](https://arxiv.org/abs/2104.09864)
- [ALiBi](https://arxiv.org/abs/2108.12409)

Both *RoPE* and *ALiBi* argue that it's best to cue the LLM about sentence order directly in the self-attention algorithm as it's there that word tokens are put into relation with each other. More specifically, sentence order should be cued by modifying the $\mathbf{QK}^T$ computation.

Without going into too many details, *RoPE* notes that positional information can be encoded into query-key pairs, *e.g.* $\mathbf{q}_i$ and $\mathbf{x}_j$ by rotating each vector by an angle $\theta * i$ and $\theta * j$ respectively with $i, j$ describing each vectors positional id:

$$ \mathbf{\hat{q}}_i^T \mathbf{\hat{x}}_j = \mathbf{{q}}_i^T \mathbf{R}_{\theta, i -j} \mathbf{{x}}_j, $$

with $R_{\theta, i - j}$ being a rotational matrix. $\theta$ is *not* learned during training, but instead set to a pre-defined value that depends on the maxium input sequence length during training.

> By doing so, the propability score between $\mathbf{q}_i$ and $\mathbf{q}_j$ is only affected if $i \ne j$ and is only affected by relative distance $i - j$ regardless of each vector's specific positions $i$ and $j$.

*RoPE* is used in multiple of todays most important LLMs, such as:
- [**Falcon**](https://huggingface.co/tiiuae/falcon-40b)
- [**Llama**](https://arxiv.org/abs/2302.13971)
- [**PaLM**](https://arxiv.org/pdf/2204.02311.pdf)

As an alternative, *ALiBi* proposes a much simpler relative position encoding scheme. The relative distance input tokens have to each other is added as negative integer scaled by `m` to each query-key entry of the $\mathbf{QK}^T$ matrix right before the softmax computation.

![](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/alibi.png)

$m$ is *not* learned during training, but instead set to a pre-defined value.
As shown in the [ALiBi](https://arxiv.org/abs/2108.12409) paper, this simple approach to relative positional encoding works very well in practice and scales well to long text input sequences.

*ALiBi* is used in multiple of todays most important LLMs, such as:
- [**MPT**](https://huggingface.co/mosaicml/mpt-30b)
- [**BLOOM**](https://huggingface.co/bigscience/bloom)

Both *RoPE* and *ALiBi* position encodings can extrapolated to input lengths not seen during training where is it has been shown to work much better out-of-the-box for ALiBi as compared to *RoPE*.
For ALiBi, one simply increases the values of the lower triangular position matrix to match the length of tinput sequence.
For *RoPE*, it has been shown that using the same $\theta$ that was used during training leads to poor results when passing text inputs much longer than those seen during training, *c.f* [Press et al.](https://arxiv.org/abs/2108.12409). However, the community has found a couple effective tricks that adapt $\theta$ that allow *RoPE* position embeddings to be extrapolated (see [here](https://github.com/huggingface/transformers/pull/24653)).

> Both RoPE and ALiBi are relative positional embeddings that are *not* learned during training, but instead are based on the following intuitions:
- Positional cues about the text inptus should be given directly to the $QK^T$ matrix of the self-attention layer
- The LLM should be incentivized to learn a constant *relative* distance positional encondings have to each other
- The further text input tokens are from each other, the lower should be the probability of their query-value probability. Both RoPE and ALiBi lower the query-key probability of tokens far away from each other. RoPE by decreasing their vector product by increasing the angle between the query-key vectors. ALiBi by adding large negative numbers to the vector product

As a conclusion, LLMs that are intended to be deployed in tasks that require to handle large text inputs are better to be trained with relative positional embeddings, such as RoPE and ALiBi. Also note that even if a LLM with RoPE and ALiBi has been trained only on a fixed length of say $N_1 = 2048$ it can still be used in practice with text inputs much larger than $N_1$, like $N_2 = 8192 > N_1$ by extrapolating the positional embeddings.

### 3.2 The k/v cache

Auto-regressive text generation with LLMs works by iteratively putting in an input sequence, sampling the next token, appending the next token to the input sequence and continuing to do so until the LLM produces a token that signifies that the generation has finished.

Please have a look at [Transformer's Generate Text Tutorial](https://huggingface.co/docs/transformers/llm_tutorial#generate-text) to get a more visual explanation for how auto-regressive generation works.

Let's run a quick code snippet to show how auto-regressive works in practice. We will simply take the most likely next token via `torch.argmax`.

In [ ]:
input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"]

for _ in range(5):
  next_logits = model(input_ids)["logits"][:, -1]
  next_token_id = torch.argmax(next_logits)

  input_ids = torch.cat([input_ids, next_token_id])
  print("shape of input_ids", input_ids.shape)

generated_text = tokenizer.batch_decode(input_ids[:, -5:])
generated_text

As we can see every time we increase the text input tokens by the just sampled token.

With very few expections, LLMs are trained using the [causal language modeling objective](https://huggingface.co/docs/transformers/tasks/language_modeling#causal-language-modeling) and therefore mask the upper triangle matrix of the attention score - this is why in the two diagrams above the attention scores are left blank (*a.k.a* have 0 probability). For a quick recap on causal language modeling you can refer to the [*Illustrated Self Attention blog*](https://jalammar.github.io/illustrated-gpt2/#part-2-illustrated-self-attention).

As a consequence, tokens *never* depend on previous tokens, more specifically the $\mathbf{q}_i$ vector is never put in relation with any key, values vectors $\mathbf{k}_j, \mathbf{v}_j$ if $j > i$. Instead $\mathbf{q}_i$ only attends to previous key-value vectors $\mathbf{k}_{m < i}, \mathbf{v}_{m < i} \text{ , for } m \in \{0, \ldots i - 1\}$. In order to reduce unnecessary computation, one can therefore cache the k/v vectors of all previous timesteps and all previous layers.

In Transformers, we can easialy retrieve the k/v cache by passing the `use_cache` flag to the `forward` call.

In [ ]:
past_key_values = None # past_key_values is the k/v cache
generated_tokens = []

for _ in range(5):
  next_logits, past_key_values = model(input_ids, past_key_values=past_key_values, use_cache=True).to_tuple()
  input_ids = torch.argmax(next_logits)

  print("shape of input_ids", input_ids.shape)
  print("length of k/v cache", len(past_key_values[0][0][1]))  # past_key_values are of shape [num_layers, 0 for k, 1 for v, batch_size, length, hidden_dim]
  generated_tokens.append(input_ids)

generated_text = tokenizer.batch_decode(generated_tokens)
generated_text

This time, the text input tokens are *not* increased, but stay constant. Instead, every time the length of the key-value cache is increased by one corresponding to the new cached key-value states of every layer.

> Making use of the key-value cache means that the $\mathbf{QK}^T$ is essentially reduced to $\mathbf{q}_c\mathbf{K}^T$ with $\mathbf{q}_c$ being the query projection of the currently passed input token which is *always* just a single vector.

This has two advantages:
- Significant increase in computational efficiency as much fewer computations are done compared to computing the full $\mathbf{QK}^T$ matrix. This leads to a big increase in inference speed
- The maximum required memory is not increased quadratically with the number of generated tokens, but only linearly.

> In short, one should *always* make use of the k/v cache as it leads to identical results and a significant speed-up for longer input sequences. Transformers has the k/v cache enabled by default when making use the text pipeline or the [`generate` method](https://huggingface.co/docs/transformers/main_classes/text_generation).


There is however one catch. While the required peak memory for the $\mathbf{QK}^T$ matrix is significantly reduced, just holding the k/v cache in memory becomes also very expensive, the longer the input sequence. Remember that the k/v cache needs to hold the key-value vectors for all previous input vectors $\mathbf{x}_i \text{, for } i \in \{1, \ldots, c - 1\}$ for all self-attention layers and for all attention heads.

Let's compute the number of float values that need to be stored in the k/v cache for our LLM.

$$ 2 \times (\text{seq_len} - 1) \times \text{num_attn_heads} \times \text{attn_head_dim} \times \text{num_layers} $$

Computing this for our LLM at a hypothetical input sequence length of 16000 gives:

In [ ]:
config = model.config
2 * 16_000 * config.n_layer * config.n_head * config.n_embd // config.n_head

Roughly 8 billion float values! Storing 8 billion float values in float16 precision requires roughly 15 GB of RAM roughly half as much as the model weights themselves!

Researchers have proposed two methods that allow to significantly reduce the memory cost of storing the key-value cache:

- 1. [Multi-Query-Attention (MQA)](https://arxiv.org/abs/1911.02150)
- 2. [Grouped-Query-Attention (GPA)](https://arxiv.org/abs/2305.13245)

## 4. Advanced: Speculative decoding

- Link to blog post
- Link to some more papers